In [1]:
import pandas as pd
import numpy as np
import glob
import datetime
#import geopandas as gp

In [2]:
path = r'C:\Users\Erin\Documents\nss-data-analytics\projects\Capstone\mapping-mexico\data\icews' # use your path
all_files = glob.glob(path + "/*.tab")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep='\t', index_col=None, header=0, parse_dates=['Event Date'], dtype={
    'Event ID':'int','Source Name':'str', 'Source Sectors':'str',
       'Source Country':'str', 'Event Text':'str', 'CAMEO Code':'str', 'Intensity':'float',
       'Target Name':'str', 'Target Sectors':'str', 'Target Country':'str', 'Story ID':'int',
       'Sentence Number':'int', 'Publisher':'str', 'City':'str', 'District':'str', 'Province':'str',
       'Country':'str', 'Latitude':'object', 'Longitude':'object'})
    li.append(df)

icews_clean = pd.concat(li, axis=0, ignore_index=True)

In [3]:
icews_clean.head()

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,926685,1995-01-01,Extremist (Russia),"Radicals / Extremists / Fundamentalists,Dissident",Russian Federation,Praise or endorse,051,3.4,Boris Yeltsin,"Elite,Executive,Executive Office,Government",Russian Federation,28235806,5,The Toronto Star,Moscow,NaN,Moskva,Russian Federation,55.7522,37.6156
1,926687,1995-01-01,Government (Bosnia and Herzegovina),Government,Bosnia and Herzegovina,Express intent to cooperate,030,4.0,Citizen (Serbia),"General Population / Civilian / Social,Social",Serbia,28235807,1,The Toronto Star,NaN,NaN,Bosnia,Bosnia and Herzegovina,44,18
2,926686,1995-01-01,Citizen (Serbia),"General Population / Civilian / Social,Social",Serbia,Express intent to cooperate,030,4.0,Government (Bosnia and Herzegovina),Government,Bosnia and Herzegovina,28235807,1,The Toronto Star,NaN,NaN,Bosnia,Bosnia and Herzegovina,44,18
3,926688,1995-01-01,Canada,NaN,Canada,Praise or endorse,051,3.4,City Mayor (Canada),"Government,Local,Municipal",Canada,28235809,3,The Toronto Star,NaN,NaN,Ontario,Canada,49.2501,-84.4998
4,926689,1995-01-01,Lawyer/Attorney (Canada),"Legal,Social",Canada,"Arrest, detain, or charge with legal action",173,-5.0,Police (Canada),"Government,Police",Canada,28235964,1,The Toronto Star,Montreal,Montreal,Quebec,Canada,45.5088,-73.5878


In [5]:
icews_clean = icews_clean.rename({'Event ID':'event_id', 'Event Date':'event_date', 'Source Name':'source_name',
    'Source Sectors':'source_sectors','Source Country':'source_country', 'Event Text':'event_text',
    'CAMEO Code':'cameo_code', 'Intensity':'intensity','Target Name':'target_name','Target Sectors':'target_sectors',
    'Target Country':'target_country', 'Story ID':'story_id', 'Sentence Number':'sentence_number',
    'Publisher':'publisher', 'City':'city', 'District':'district', 'Province':'provice',
    'Country':'country', 'Latitude':'lat', 'Longitude':'lon'})

In [7]:
mex_clean = icews_clean.isin('Mexico').all(1)

TypeError: only list-like or dict-like objects are allowed to be passed to DataFrame.isin(), you passed a 'str'